# Get dataset

In [1]:
import pandas as pd
import numpy as np
import random
import re  

## Templates

In [2]:
# templates
templates = pd.read_json("data/templates.json")
init_uttr_updated = pd.read_csv("data/InitUtterance_list_updated.csv")
templates_updated = templates.merge(init_uttr_updated, on='id')
templates_updated = templates_updated[['id','de', 'fr', 'it', 'en']]
templates_updated.head()

id  \
0       MyCloud.OpenArea.Init.Utterance   
1    Router.SwitchOnWiFi.Init.Utterance   
2     Tv.GenericExitToTv.Init.Utterance   
3   Router.SwitchOffWiFi.Init.Utterance   
4  Tv.RadioChannelChange.Init.Utterance   

                                                  de  \
0  {'texts': [{'ttsText': '{MyCloudArea} auf #myC...   
1  {'texts': [{'ttsText': 'aktiviere das Wlan'}, ...   
2  {'texts': [{'ttsText': 'gehe zurück zum Fernse...   
3  {'texts': [{'ttsText': 'deaktiviere das Wlan'}...   
4  {'texts': [{'ttsText': 'auf Radio {RadioChanne...   

                                                  fr  \
0  {'texts': [{'ttsText': 'afficher {MyCloudArea}...   
1  {'texts': [{'ttsText': 'active le Wi-Fi'}, {'t...   
2  {'texts': [{'ttsText': 'revenir à la télé'}, {...   
3  {'texts': [{'ttsText': 'désactiver le Wi-Fi'},...   
4  {'texts': [{'ttsText': 'prendre radio {RadioCh...   

                                                  it  \
0  {'texts': [{'ttsText': 'apri {MyCloudArea} su ...   
1  {'texts': [{'ttsText': 'attiva il #wifi'}, {'t...   
2  {'texts': [{'ttsText': 'torna alla TV'}, {'tts...   
3  {'texts': [{'ttsText': 'disattiva il wifi'}, {...   
4  {'texts': [{'ttsText': 'metti radio {RadioChan...   

                                                  en  
0  {'texts': [{'ttsText': 'show {MyCloudArea} on ...  
1  {'texts': [{'ttsText': 'activate the Wi-Fi'}, ...  
2  {'texts': [{'ttsText': 'back to TV'}, {'ttsTex...  
3  {'texts': [{'ttsText': 'can you please switch ...  
4  {'texts': [{'ttsText': 'can you turn on radio ...

In [3]:
templates_updated.describe()

id  \
count                             39   
unique                            39   
top     Tv.OpenUiArea.Init.Utterance   
freq                               1   

                                                       de  \
count                                                  39   
unique                                                 39   
top     {'texts': [{'ttsText': '{SmartHomeSceneName} S...   
freq                                                    1   

                                                       fr  \
count                                                  39   
unique                                                 39   
top     {'texts': [{'ttsText': 'activer scène {SmartHo...   
freq                                                    1   

                                                       it  \
count                                                  39   
unique                                                 39   
top     {'texts': [{'ttsText': 'attiva scena {SmartHom...   
freq                                                    1   

                                                       en  
count                                                  39  
unique                                                 39  
top     {'texts': [{'ttsText': 'activate {SmartHomeSce...  
freq                                                    1

## Entities

In [4]:
# entities
entities = pd.read_json("data/entities.json")
entities = entities[['value', 'type', 'language', 'normalizedValue', 'aliases']]
entities.head()

value                 type language normalizedValue aliases
0     Widnau  LocalsearchLocation       de          widnau      []
1     Zurigo  LocalsearchLocation       it          zurigo      []
2  Solothurn  LocalsearchLocation       de       solothurn      []
3    Muttenz  LocalsearchLocation       de         muttenz      []
4       Lyss  LocalsearchLocation       de            lyss      []

In [5]:
entities.describe()

value        type language normalizedValue aliases
count   65471       65471    65471           65471   65471
unique  51272          33        4           52082    3793
top      Golf  SeriesName       de            golf      []
freq       20       17913    22746              20   59649

# Explore data

### Entities catagorical count

In [6]:
df = entities[['type', 'language']]
df = df.groupby(['type','language']).size().reset_index(name='count') 
df.pivot(index='type', columns='language', values='count')#.plot(kind='bar')

language                     de      en      fr      it
type                                                   
AlbumName                   NaN  1479.0     NaN     NaN
AppName                   326.0   273.0   312.0   290.0
AreaName                   26.0    26.0    26.0    26.0
ArtistName                  NaN   822.0     NaN     NaN
BroadcastName             715.0   483.0   676.0   663.0
CallingNumberType           3.0     4.0     4.0     4.0
ClipName                  353.0   320.0   342.0   338.0
DossierName                 8.0     3.0     5.0     3.0
FirstName                 952.0     NaN     NaN     NaN
GenreName                   NaN     8.0     NaN     NaN
LastName                  998.0     NaN     NaN     NaN
LocalsearchLocation      3403.0    28.0  1909.0  1572.0
MyCloudArea                 5.0     5.0     5.0     5.0
NickName                   20.0    25.0    21.0    20.0
ParticipantName          2710.0   297.0  1473.0   575.0
RadioChannelName          366.0   365.0   365.0   365.0
SeriesName               6186.0  3281.0  4766.0  3680.0
SmartHomeDeviceBrand        8.0    10.0    10.0     9.0
SmartHomeDeviceCategory     5.0     5.0     5.0     5.0
SmartHomeDeviceLocation     9.0     9.0     9.0     9.0
SmartHomeDeviceName       100.0   100.0   100.0   101.0
SmartHomeSceneName         50.0    49.0    50.0    50.0
SportCompetitionName       20.0    20.0    20.0    20.0
SportEventName            206.0   207.0   203.0   207.0
SportParticipantName      432.0   432.0   432.0   432.0
SportSportsName            10.0    10.0    10.0    10.0
TrackName                   NaN  2701.0     NaN     NaN
TvChannelName             635.0   623.0   633.0   633.0
TvGenre                   137.0   131.0   131.0   127.0
VodGenre                   99.0   107.0    99.0    99.0
VodName                  4961.0  4209.0  3760.0  2083.0
entitytype_1                1.0     NaN     NaN     NaN
test_aliasesTest            2.0     NaN     1.0     NaN

### Generate correct command

In [25]:
from generator.module import Generator, Normalizer
gen = Generator(templates=templates_updated, entities=entities)
gen.get_command(target_id='Tv.RadioChannelChange.Init.Utterance', 
                target_lang='de', 
                normalizer= Normalizer().normalize_text,
                verbose=True)

Choose template: 
	stell auf Radio {RadioChannelName}
After tag removal: 
	stell auf Radio radio austria
After normalizer: 
	stell auf radio radio austria


'stell auf radio radio austria'

In [19]:

gen.get_command(target_id='MyCloud.OpenArea.Init.Utterance', 
                    target_lang='fr', 
                    normalizer= Normalizer().normalize_text,
                    verbose=True)

Choose template: 
	tu peux m'afficher {MyCloudArea} sur #myCloud s'il te plaît
After tag removal: 
	tu peux m'afficher albums sur #myCloud s'il te plaît
After normalizer: 
	tu peux m'afficher albums sur mycloud s'il te plaît


"tu peux m'afficher albums sur mycloud s'il te plaît"

In [17]:
normalizer = 
gen = Generator(templates=templates_updated, entities=entities)

gen.generate_command(target_id='Tv.GenericExitToTv.Init.Utterance', 
                    target_lang='de', 
                    normalizer= normalizer)

'wieder fernsehen'

In [ ]:
pd.DataFrame()